In [1]:
from PIL import Image
import os
import os.path as osp
import numpy as np
import torch
import matplotlib.pyplot as plt

# Select you GPU
I_GPU = 0

# Uncomment to use autoreload
%load_ext autoreload
%autoreload 2

import os
import os.path as osp
import sys
import torch
import numpy as np
from time import time
from omegaconf import OmegaConf
from PIL import Image
start = time()
import warnings
warnings.filterwarnings('ignore')

# torch.cuda.set_device(I_GPU)
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

from torch_points3d.utils.config import hydra_read
from torch_geometric.data import Data
from torch_points3d.core.multimodal.data import MMData, MMBatch
from torch_points3d.visualization.multimodal_data import visualize_mm_data
from torch_points3d.core.multimodal.image import SameSettingImageData, ImageData
from torch_points3d.datasets.segmentation.multimodal.scannet import ScannetDatasetMM
from torch_points3d.datasets.segmentation.scannet import CLASS_COLORS, CLASS_NAMES, CLASS_LABELS
from torch_points3d.metrics.segmentation_tracker import SegmentationTracker

from pykeops.torch import LazyTensor

import plotly.io as pio

#pio.renderers.default = 'jupyterlab'        # for local notebook
pio.renderers.default = 'iframe_connected'  # for remote notebook. Other working (but seemingly slower) options are: 'sphinx_gallery' and 'iframe'

CLASS_COLORS[0] = (174.0, 199.0, 232.0)
CLASS_COLORS[-1] = (0, 0, 0)

# from torch_points3d.datasets.segmentation.scannet import CLASS_COLORS

MMData debug() function changed, please uncomment the 3rd assert line when doing inference without M2F features!


In [2]:
import os
import torchnet as tnt
import torch
from typing import Dict, Any
import wandb
from torch.utils.tensorboard import SummaryWriter
import logging
from torch_points3d.metrics.confusion_matrix import ConfusionMatrix
from torch_points3d.models import model_interface
from torch_points3d.metrics.base_tracker import BaseTracker, meter_value
from torch_points3d.metrics.meters import APMeter
from torch_points3d.datasets.segmentation import IGNORE_LABEL

from torch_geometric.nn.unpool import knn_interpolate
from torch_points3d.core.data_transform import SaveOriginalPosId

log = logging.getLogger(__name__)


def meter_value(meter, dim=0):
    return float(meter.value()[dim]) if meter.n > 0 else 0.0


class BaseTracker:
    def __init__(self, stage: str, wandb_log: bool, use_tensorboard: bool):
        self._wandb = wandb_log
        self._use_tensorboard = use_tensorboard
        self._tensorboard_dir = os.path.join(os.getcwd(), "tensorboard")
        self._n_iter = 0
        self._finalised = False
        self._conv_type = None

        if self._use_tensorboard:
            log.info(
                "Access tensorboard with the following command <tensorboard --logdir={}>".format(self._tensorboard_dir)
            )
            self._writer = SummaryWriter(log_dir=self._tensorboard_dir)

    def reset(self, stage="train"):
        self._stage = stage
        self._loss_meters = {}
        self._finalised = False

    def get_metrics(self, verbose=False) -> Dict[str, Any]:
        metrics = {}
        for key, loss_meter in self._loss_meters.items():
            value = meter_value(loss_meter, dim=0)
            if value:
                metrics[key] = meter_value(loss_meter, dim=0)
        return metrics

    @property
    def metric_func(self):
        self._metric_func = {"loss": min}
        return self._metric_func

    def track(self, model: model_interface.TrackerInterface, **kwargs):
        if self._finalised:
            raise RuntimeError("Cannot track new values with a finalised tracker, you need to reset it first")
            
        if model is not None:
            losses = self._convert(model.get_current_losses())
            self._append_losses(losses)

    def finalise(self, *args, **kwargs):
        """ Lifcycle method that is called at the end of an epoch. Use this to compute
        end of epoch metrics.
        """
        self._finalised = True

    def _append_losses(self, losses):
        for key, loss in losses.items():
            if loss is None:
                continue
            loss_key = "%s_%s" % (self._stage, key)
            if loss_key not in self._loss_meters:
                self._loss_meters[loss_key] = tnt.meter.AverageValueMeter()
            self._loss_meters[loss_key].add(loss)

    @staticmethod
    def _convert(x):
        if torch.is_tensor(x):
            return x.detach().cpu().numpy()
        else:
            return x

    def publish_to_tensorboard(self, metrics, step):
        for metric_name, metric_value in metrics.items():
            metric_name = "{}/{}".format(metric_name.replace(self._stage + "_", ""), self._stage)
            self._writer.add_scalar(metric_name, metric_value, step)

    @staticmethod
    def _remove_stage_from_metric_keys(stage, metrics):
        new_metrics = {}
        for metric_name, metric_value in metrics.items():
            new_metrics[metric_name.replace(stage + "_", "")] = metric_value
        return new_metrics

    def publish(self, step):
        """ Publishes the current metrics to wandb and tensorboard
        Arguments:
            step: current epoch
        """
        metrics = self.get_metrics()

        if self._wandb:
            wandb.log(metrics, step=step)

        if self._use_tensorboard:
            self.publish_to_tensorboard(metrics, step)

        # Some metrics may be intended for wandb or tensorboard
        # tracking but not for final final model selection. Those are
        # the metrics absent from self.metric_func and must be excluded
        # from the output of self.publish
        current_metrics = {
            k: v
            for k, v in self._remove_stage_from_metric_keys(self._stage, metrics).items()
            if k in self.metric_func.keys()}

        return {
            "stage": self._stage,
            "epoch": step,
            "current_metrics": current_metrics,
        }

    def print_summary(self):
        metrics = self.get_metrics(verbose=True)
        log.info("".join(["=" for i in range(50)]))
        for key, value in metrics.items():
            log.info("    {} = {}".format(key, value))
        log.info("".join(["=" for i in range(50)]))

    @staticmethod
    def _dict_to_str(dictionnary):
        string = "{"
        for key, value in dictionnary.items():
            string += "%s: %.2f," % (str(key), value)
        string += "}"
        return string


class SegmentationTracker(BaseTracker):
    def __init__(
        self, dataset, stage="train", wandb_log=False, use_tensorboard: bool = False, ignore_label: int = IGNORE_LABEL
    ):
        """ This is a generic tracker for multimodal tasks.
        It uses a confusion matrix in the back-end to track results.
        Use the tracker to track an epoch.
        You can use the reset function before you start a new epoch

        Arguments:
            dataset  -- dataset to track (used for the number of classes)

        Keyword Arguments:
            stage {str} -- current stage. (train, validation, test, etc...) (default: {"train"})
            wandb_log {str} --  Log using weight and biases
        """
        super(SegmentationTracker, self).__init__(stage, wandb_log, use_tensorboard)
        self._num_classes = dataset.num_classes
        self._ignore_label = ignore_label
        self._dataset = dataset
        self.reset(stage)
        self._metric_func = {
            "miou": max,
            "macc": max,
            "acc": max,
            "loss": min,
            "map": max,
        }  # Those map subsentences to their optimization functions

    def reset(self, stage="train"):
        super().reset(stage=stage)
        self._confusion_matrix = ConfusionMatrix(self._num_classes)
        self._acc = 0
        self._macc = 0
        self._miou = 0
        self._miou_per_class = {}

    @staticmethod
    def detach_tensor(tensor):
        if torch.torch.is_tensor(tensor):
            tensor = tensor.detach()
        return tensor

    @property
    def confusion_matrix(self):
        return self._confusion_matrix.confusion_matrix

    def track(self, model: model_interface.TrackerInterface, pred_labels=None, gt_labels=None, **kwargs):
        """ Add current model predictions (usually the result of a batch) to the tracking
        """
        if not self._dataset.has_labels(self._stage):
            return

        # Feng: to evaluate M2F predictions instead of model logits
        if pred_labels is not None and gt_labels is not None:
            outputs = pred_labels
            targets = gt_labels
        else:
            super().track(model)
            
            outputs = model.get_output()
            targets = model.get_labels()
        self._compute_metrics(outputs, targets)

    def _compute_metrics(self, outputs, labels):
        mask = labels != self._ignore_label
        outputs = outputs[mask]
        labels = labels[mask]

        outputs = self._convert(outputs)
        labels = self._convert(labels)

        if len(labels) == 0:
            return

        assert outputs.shape[0] == len(labels)
        
        # Check if output is predicted label or logits
        if len(outputs.shape) > 1:
            self._confusion_matrix.count_predicted_batch(labels, np.argmax(outputs, 1))
        else:
            
            self._confusion_matrix.count_predicted_batch(labels, outputs)

        self._acc = 100 * self._confusion_matrix.get_overall_accuracy()
        self._macc = 100 * self._confusion_matrix.get_mean_class_accuracy()
        self._miou = 100 * self._confusion_matrix.get_average_intersection_union()
        self._miou_per_class = {
            i: "{:.2f}".format(100 * v)
            for i, v in enumerate(self._confusion_matrix.get_intersection_union_per_class()[0])
        }

    def get_metrics(self, verbose=False) -> Dict[str, Any]:
        """ Returns a dictionnary of all metrics and losses being tracked
        """
        metrics = super().get_metrics(verbose)

        metrics["{}_acc".format(self._stage)] = self._acc
        metrics["{}_macc".format(self._stage)] = self._macc
        metrics["{}_miou".format(self._stage)] = self._miou

        if verbose:
            metrics["{}_miou_per_class".format(self._stage)] = self._miou_per_class
        return metrics

    @property
    def metric_func(self):
        return self._metric_func


class ScannetSegmentationTracker(SegmentationTracker):
    def reset(self, stage="train"):
        super().reset(stage=stage)
        self._full_confusion_matrix = ConfusionMatrix(self._num_classes)
        self._raw_datas = {}
        self._votes = {}
        self._vote_counts = {}
        self._full_preds = {}
        self._full_acc = None

    def track(self, model: model_interface.TrackerInterface, full_res=False, pred_labels=None, gt_labels=None, **kwargs):
        """ Add current model predictions (usually the result of a batch) to the tracking
        """
        if pred_labels is not None and gt_labels is not None:
            super().track(model=None, pred_labels=pred_labels, gt_labels=gt_labels)
        else:
            super().track(model)

            # Set conv type
            self._conv_type = model.conv_type

            # Train mode or low res, nothing special to do
            if not full_res or self._stage == "train" or kwargs.get("data") is None:
                return

            data = kwargs.get("data", model.get_input())
            data = data.data if model.is_multimodal else data
            self._vote(data, model.get_output())

    def get_metrics(self, verbose=False) -> Dict[str, Any]:
        """ Returns a dictionnary of all metrics and losses being tracked
        """
        metrics = super().get_metrics(verbose)
        if self._full_acc:
            metrics["{}_full_acc".format(self._stage)] = self._full_acc
            metrics["{}_full_macc".format(self._stage)] = self._full_macc
            metrics["{}_full_miou".format(self._stage)] = self._full_miou
        return metrics

    def finalise(self, full_res=False, make_submission=False, **kwargs):
        if not full_res and not make_submission:
            return
        
        self._predict_full_res()

        # Compute full res metrics
        if self._dataset.has_labels(self._stage):
            for scan_id in self._full_preds:
                full_labels = self._raw_datas[scan_id].y
                # Mask ignored labels
                mask = full_labels != self._ignore_label
                full_labels = full_labels[mask]
                full_preds = self._full_preds[scan_id].cpu()[mask].numpy()
                self._full_confusion_matrix.count_predicted_batch(full_labels, full_preds)

            self._full_acc = 100 * self._full_confusion_matrix.get_overall_accuracy()
            self._full_macc = 100 * self._full_confusion_matrix.get_mean_class_accuracy()
            self._full_miou = 100 * self._full_confusion_matrix.get_average_intersection_union()
            
        # Save files to disk
        if make_submission and self._stage == "test":
            self._make_submission()

    def _make_submission(self):
        orginal_class_ids = np.asarray(self._dataset.train_dataset.valid_class_idx)
        path_to_submission = self._dataset.path_to_submission
        for scan_id in self._full_preds:
            full_pred = self._full_preds[scan_id].cpu().numpy().astype(np.int8)
            full_pred = orginal_class_ids[full_pred]  # remap labels to original labels between 0 and 40
            scan_name = self._raw_datas[scan_id].scan_name
            path_file = osp.join(path_to_submission, "{}.txt".format(scan_name))
            np.savetxt(path_file, full_pred, delimiter="/n", fmt="%d")

    def _vote(self, data, output):
        """ Populates scores for the points in data

        Parameters
        ----------
        data : Data
            should contain `pos` and `SaveOriginalPosId.KEY` keys
        output : torch.Tensor
            probablities out of the model, shape: [N,nb_classes]
        """
        id_scans = data.id_scan
        if id_scans.dim() > 1:
            id_scans = id_scans.squeeze()
        if self._conv_type == "DENSE":
            batch_size = len(id_scans)
            output = output.view(batch_size, -1, output.shape[-1])

        for idx_batch, id_scan in enumerate(id_scans):
            # First time we see this scan
            if id_scan not in self._raw_datas:
                raw_data = self._dataset.get_raw_data(self._stage, id_scan, remap_labels=True)
                self._raw_datas[id_scan] = raw_data
                self._vote_counts[id_scan] = torch.zeros(raw_data.pos.shape[0], dtype=torch.int)
                self._votes[id_scan] = torch.zeros((raw_data.pos.shape[0], self._num_classes), dtype=torch.float)
            else:
                raw_data = self._raw_datas[id_scan]

            batch_mask = idx_batch
            if self._conv_type != "DENSE":
                batch_mask = data.batch == idx_batch
            idx = data[SaveOriginalPosId.KEY][batch_mask]

            self._votes[id_scan][idx] += output[batch_mask].cpu()
            self._vote_counts[id_scan][idx] += 1

    def _predict_full_res(self):
        """ Predict full resolution results based on votes """
        for id_scan in self._votes:
            has_prediction = self._vote_counts[id_scan] > 0
            self._votes[id_scan][has_prediction] /= self._vote_counts[id_scan][has_prediction].unsqueeze(-1)

            # Upsample and predict
            full_pred = knn_interpolate(
                self._votes[id_scan][has_prediction],
                self._raw_datas[id_scan].pos[has_prediction],
                self._raw_datas[id_scan].pos,
                k=1,
            )
            self._full_preds[id_scan] = full_pred.argmax(-1)


In [3]:
# Set your dataset root directory, where the data was/will be downloaded
DATA_ROOT = '/scratch-shared/fsun/dvata'

dataset_config = 'segmentation/multimodal/Feng/scannet-neucon-smallres-m2f'   
models_config = 'segmentation/multimodal/Feng/mvfusion'    # model family
model_name = 'MVFusion_3D_small_6views'                       # specific model

overrides = [
    'task=segmentation',
    f'data={dataset_config}',
    f'models={models_config}',
    f'model_name={model_name}',
    f'data.dataroot={DATA_ROOT}',
]

cfg = hydra_read(overrides)
OmegaConf.set_struct(cfg, False)  # This allows getattr and hasattr methods to function correctly
cfg.data.load_m2f_masks = True   # load Mask2Former predicted masks

cfg.data.m2f_preds_dirname = 'ViT_masks'
cfg.data.n_views = 6 #cfg.models[model_name].backbone.transformer.n_views
print(cfg.data.n_views)

# Dataset instantiation
start = time()
dataset = ScannetDatasetMM(cfg.data)
# print(dataset)
print(f"Time = {time() - start:0.1f} sec.")

6
Load predicted 2D semantic segmentation labels from directory  ViT_masks
initialize train dataset
initialize val dataset
Time = 7.5 sec.


In [4]:
tracker_mvfusion = ScannetSegmentationTracker(dataset=dataset, stage='train', wandb_log=False, use_tensorboard=False, ignore_label=IGNORE_LABEL)
tracker_m2f = ScannetSegmentationTracker(dataset=dataset, stage='train', wandb_log=False, use_tensorboard=False, ignore_label=IGNORE_LABEL)


# Temporal Consistency using optical flow

In [5]:
from mmflow.apis import init_model, inference_model
from mmflow.datasets import visualize_flow, write_flow
import mmcv
from mmflow.ops import Warp

# Specify the path to model config and checkpoint file
# config_file = '/home/fsun/DeepViewAgg/flow/configs/pwcnet_ft_4x1_300k_sintel_final_384x768.py'
# checkpoint_file = '/home/fsun/DeepViewAgg/flow/pretrained/pwcnet_ft_4x1_300k_sintel_final_384x768.pth'

config_file = '/home/fsun/DeepViewAgg/flow/configs/flownet2_8x1_sfine_flyingthings3d_subset_384x768.py'
checkpoint_file = '/home/fsun/DeepViewAgg/flow/pretrained/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth'

# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

warp = Warp(mode='nearest',
            padding_mode='zeros',
            align_corners=False,
            use_mask=True).cuda()

2023-01-11 22:40:23,677 - mmflow - INFO - Freeze the parameters in FlowNetCSS
2023-01-11 22:40:24,687 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /home/fsun/DeepViewAgg/flow/pretrained/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth


In [26]:
scan_dir = "/scratch-shared/fsun/data/scannet/scans"

scene_id = 'scene0000_00'
scene_dir = osp.join(scan_dir, scene_id)

mask_im_dir = osp.join('/home/fsun/data/scannet/scans', scene_id, 'ViT_masks')
# mask_im_dir = osp.join('/scratch-shared/fsun/data/scannet/scans', scene_id, 'color_resized')


im_ids = sorted(os.listdir(mask_im_dir), key=lambda i: int(os.path.splitext(os.path.basename(i))[0]))


tracker_m2f.reset(stage='train')
tracker_mvfusion.reset(stage='train')

for i in range(len(im_ids)-1):
    
    im1_p = osp.join(scene_dir, 'color_resized', im_ids[i])
    im2_p = osp.join(scene_dir, 'color_resized', im_ids[i+1])
    
    # compute flow map from im1 to im2
    result = inference_model(model, im2_p, im1_p)
    flow_map = torch.tensor(result).permute(2, 0, 1).unsqueeze(0)
    
#     im1 = to_tensor(Image.open(im1_p)).unsqueeze(0)
#     im2 = to_tensor(Image.open(im1_p)).unsqueeze(0)

#     # warps im1 to im2
#     im_warped = warp(im1, flow_map).permute(0, 2, 3, 1)[0]

#     plt.imshow(Image.open(im1_p))
#     plt.show()
#     plt.imshow(Image.open(im2_p))
#     plt.show()
#     plt.imshow(im_warped)
#     plt.show()

    
    # img loading
    seg_im_p1 = osp.join(mask_im_dir, im_ids[i])
    seg_im_p2 = osp.join(mask_im_dir, im_ids[i+1])
    seg_im1 = np.asarray(Image.open(seg_im_p1)) 
    seg_im2 = np.asarray(Image.open(seg_im_p2)).astype(np.int) - 1   # Adjust labels
    
    
    # warping im1 to im2
    seg_im1_semantic = torch.tensor(seg_im1).unsqueeze(0).unsqueeze(0).float()
    seg_im_warped = warp(seg_im1_semantic, flow_map).permute(0, 2, 3, 1)[0].squeeze() - 1    # Adjust labels

    # take im2 as 'pseudo gt' for temporal consistency. Thus, invalidly warped pixels should be ignored
    # by setting the corresponding gt label to the IGNORE_LABEL
    seg_im2[seg_im_warped == -1] = -1
    tracker_m2f.track(pred_labels=seg_im_warped.long(), gt_labels=seg_im2, model=None)
#     tracker_mvfusion.track(pred_labels=mm_data.data.pred, gt_labels=mm_data.data.y, model=None)

    
    seg_im1_rgb = np.array(CLASS_COLORS)[seg_im_warped.long()]
    seg_im1_rgb = Image.fromarray(seg_im1_rgb.astype('uint8'))
    
    seg_im2_rgb = np.array(CLASS_COLORS)[seg_im2]
    seg_im2_rgb = Image.fromarray(seg_im2_rgb.astype('uint8'))
    

#     plt.imshow(seg_im1_rgb)
#     plt.show()
#     plt.imshow(seg_im2_rgb)
#     plt.show()

    
#     if i == 3:
#         break
        
tracker_m2f.get_metrics()

{'train_acc': 95.78605746787561,
 'train_macc': 80.25658729487196,
 'train_miou': 72.34121676818701}